In [1]:
import utils
import loader
import torch
path_HE = 'C:/Users/phili/OneDrive/Uni/WS_22/Masterarbeit/Masterarbeit_Code_Philipp_Rosin/Data_set_BCI_challange/train/HE'
path_IHC = 'C:/Users/phili/OneDrive/Uni/WS_22/Masterarbeit/Masterarbeit_Code_Philipp_Rosin/Data_set_BCI_challange/train/IHC'

params = utils.get_config_from_yaml('C:/Users/phili/OneDrive/Uni/WS_22/Masterarbeit/Masterarbeit_Code_Philipp_Rosin/torch_stain_transfer/code/config.yaml')
k=1

train_data = loader.stain_transfer_dataset( img_patch=  k,
                                            params= params,
                                            HE_img_dir = path_HE,
                                            IHC_img_dir = path_IHC,                                                     
                                           )

real_HE, real_IHC,img_name = train_data[4]
real_HE = torch.unsqueeze(real_HE,0)            

c:\Users\phili\OneDrive\Uni\WS_22\Masterarbeit\Masterarbeit_Code_Philipp_Rosin\torch_stain_transfer\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import trasnformer_gan

trans_gen = trasnformer_gan.Discriminator(
                                      chw = [3,256,256], 
                                      patch_size = [32, 32],
                                      num_heads = 2, 
                                      num_blocks = 2)
trans_gen.cuda()


c:\Users\phili\OneDrive\Uni\WS_22\Masterarbeit\Masterarbeit_Code_Philipp_Rosin\torch_stain_transfer\code\trasnformer_gan.py:160: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.pos_embedding = nn.Parameter(torch.tensor(get_positional_Embeddings((patch_size[0]**2)* chw[0] , self.embedding_dim)))


Generator(
  (patch_embedding): patch_embedding(
    (patcher): Conv2d(3, 3072, kernel_size=(32, 32), stride=(32, 32))
    (flatten): Flatten(start_dim=2, end_dim=3)
  )
  (blocks): ModuleList(
    (0): ViT_Block(
      (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (mhsa): MSA(
        (q_mappings): ModuleList(
          (0): Linear(in_features=32, out_features=32, bias=True)
          (1): Linear(in_features=32, out_features=32, bias=True)
        )
        (k_mappings): ModuleList(
          (0): Linear(in_features=32, out_features=32, bias=True)
          (1): Linear(in_features=32, out_features=32, bias=True)
        )
        (v_mappings): ModuleList(
          (0): Linear(in_features=32, out_features=32, bias=True)
          (1): Linear(in_features=32, out_features=32, bias=True)
        )
        (softmax): Softmax(dim=-1)
      )
      (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (0): Linear(in_features=64,

In [4]:


fake_IHC = trans_gen(real_HE)
print(fake_IHC.shape)


torch.Size([1, 3, 256, 256])
64
torch.Size([1, 3072, 64])
torch.Size([1, 3072, 64])
torch.Size([1, 3072, 64])
torch.Size([1, 3, 256, 256])


In [49]:
from torch import nn
import math
patch_size = 32
n, c, h, w = real_HE.shape

embedding_dim = (patch_size**2)* 3
# 1) the conv layer with kernel_size = stride writes out patches of the kernels the embedding_dim should be choosen as (patch_size**2)* in_channel so that no information is lost        
patcher = nn.Conv2d(in_channels=c,
                    out_channels = embedding_dim,
                    kernel_size = patch_size,
                    stride = patch_size,
                    padding = 0)
patcher.cuda()
# 2) flatten the feature map into 1D
flatten = nn.Flatten(start_dim=2, 
                    end_dim=3)

num_patches = (h/patch_size)**2

unsqueze =nn.Unflatten(2,(int(math.sqrt(num_patches)), int(math.sqrt(num_patches))))



In [50]:
conv = patcher(real_HE)
flatten = flatten(conv)
unflatten = unsqueze(flatten)
out = unflatten.reshape([1,3,256,256])
print(real_HE.shape)
print(conv.shape)
print(flatten.shape)
print(unflatten.shape)
print(out.shape)

torch.Size([1, 3, 256, 256])
torch.Size([1, 3072, 8, 8])
torch.Size([1, 3072, 64])
torch.Size([1, 3072, 8, 8])
torch.Size([1, 3, 256, 256])
